In [1]:
import os
import time
import Turbine
import mat_properties as prop
import numpy as n
import pandas as pd

os.environ["RPPREFIX"] = r"C:/Program Files (x86)/REFPROP"

RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")

water = prop.Materials_prop(
    "water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    RP=RP,
)


# таблица номинального режима
gas_streams0 = pd.read_excel("streams0.xlsx", sheet_name="gas", index_col=0)
water_streams0 = pd.read_excel("streams0.xlsx", sheet_name="water", index_col=0)
# таблица задаваемых значений (заменится на интерфейс)
gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
water_streams = pd.read_excel("streams.xlsx", sheet_name="water", index_col=0)

water_streams['DROSVD-TURBVD':'INKOND']

,T,P,H,G,S,X
DROSVD-TURBVD,508.600000,5.1990,3414.093682,63.40,NaN,NaN
ENDOFVD,195.852578,0.5490,2844.285130,63.40,NaN,NaN
DROSND-TURBND,209.800000,0.4000,2859.669112,14.76,NaN,NaN
SMESHEND,197.800000,0.5490,2848.521603,78.16,NaN,NaN
DOOTB2,131.400000,0.2542,2724.861533,78.16,NaN,NaN
DOOTB1,NaN,0.1525,2652.000000,78.16,NaN,NaN
INCND,NaN,0.1525,2652.000000,78.16,NaN,NaN
INKOND,41.510000,0.0017,2338.024560,78.16,NaN,NaN


In [2]:
calctolerance = 10**-3
calcmethod = "hybr"
KPD_SP = 0.99  # эффективность сетевых подогревателей
RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")
water = prop.Materials_prop(
    "water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    RP=RP,
)

Turb = Turbine.turbine(
    water,
    "DROSVD-TURBVD",
    "ENDOFVD",
    "DROSND-TURBND",
    "SMESHEND",
    "DOOTB2",
    "DOOTB1",
    "INCND",
    "INKOND",
    water_streams0,
    water_streams,
)

start_time = time.time()

res = Turb.calculate(diafragma=0, maxiterations=50, calctolerance=10**-5)

print("fin:--- %s сек. ---" % round((time.time() - start_time), 1))
res

fin:--- 0.0 сек. ---


In [3]:
# надо посчитать потерю давления НД (готово)
# отборы неправильно (второй отбор завышен, надопроверить исходные данные функции)
water_streams['DROSVD-TURBVD':'INKOND']

,T,P,H,G,S,X
DROSVD-TURBVD,505.067282,7.823186,3414.093682,63.40,6.755087,100.000000
ENDOFVD,193.369091,0.547107,2838.971378,63.40,6.984771,100.000000
DROSND-TURBND,204.354911,0.609107,2859.669112,14.76,6.980642,100.000000
SMESHEND,195.160342,0.547107,2842.880008,78.16,6.993133,100.000000
DOOTB2,128.819531,0.253500,2719.238314,78.16,7.053064,100.000000
DOOTB1,111.809361,0.152314,2646.804236,78.16,7.095783,97.887585
INCND,111.809361,0.152314,2646.804236,78.16,7.095783,97.887585
INKOND,41.445953,0.007974,2336.575021,78.16,7.467135,90.030513


In [4]:
water_streams0['DROSVD-TURBVD':'INKOND']

,T,P,H,G,S
DROSVD-TURBVD,508.600000,7.8450,3422.525472,63.40,NaN
ENDOFVD,195.852578,0.5490,2844.285130,63.40,NaN
DROSND-TURBND,209.800000,0.6110,2871.426945,14.76,7.003739
SMESHEND,197.800000,0.5490,2848.521603,78.16,7.003601
DOOTB2,131.400000,0.2542,2724.861533,78.16,7.065770
DOOTB1,NaN,0.1525,2652.000000,78.16,7.109000
INCND,NaN,0.1525,2652.000000,78.16,NaN
INKOND,41.510000,0.0080,2341.254319,78.16,NaN


In [5]:
Get_value='P'
Error=(water_streams['DROSVD-TURBVD':'INKOND'][Get_value]-water_streams0['DROSVD-TURBVD':'INKOND'][Get_value])/water_streams0['DROSVD-TURBVD':'INKOND'][Get_value]*100
Error

DROSVD-TURBVD   -0.278057
ENDOFVD         -0.344730
DROSND-TURBND   -0.309749
SMESHEND        -0.344730
DOOTB2          -0.275477
DOOTB1          -0.122170
INCND           -0.122170
INKOND          -0.330758
Name: P, dtype: float64

In [6]:
Turb.calculate_power()

{'Ni': 76.03555194768487,
 'Ni1': 36.462754094092595,
 'Ni2': 9.663834848206255,
 'Ni3': 24.247515467655415,
 'Ni4': 5.6614475377305995}